# 📘 Cross-Lingual Financial Document Retrieval Pipeline
Built with layout-aware PDF preprocessing, query translation, and BM25 retrieval.

In [ ]:
# 📦 Step 1: Setup Environment
!apt install poppler-utils -y
!pip install pymupdf easyocr pdf2image rank_bm25 openai
import os

In [ ]:
# 📂 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 📁 Unzip provided layout-aware pipeline
!unzip /content/drive/MyDrive/layout_pipeline_colab_demo.zip -d /content/

In [ ]:
# 🧱 Step 2: Preprocess PDFs into structured passages
import layout_pipeline.layout_segmenter_easyocr as layout
layout.process_folder(
    pdf_folder="/content/drive/MyDrive/pdfs", 
    output_jsonl="/content/structured_passages.jsonl"
)

In [ ]:
# 🌐 Step 3: Translate queries (placeholder for GPT/HF model)
import json
queries = [{"qid": "q1", "text_en": "What is the premium for travel insurance?"}]
translated = [{"qid": q["qid"], "text_zh": "旅遊保險的保費是多少？"} for q in queries]
with open("translated_queries.json", "w", encoding="utf-8") as f:
    json.dump(translated, f, ensure_ascii=False, indent=2)

### 2. Translate (GPT 3.5)

In [ ]:
# 🌐 GPT-based Query Translation
import openai
import json
from tqdm import tqdm

openai.api_key = "your-api-key-here"  # Replace with your actual API key

def translate_with_gpt(query_en, model="gpt-3.5-turbo"):
    messages = [
        {"role": "system", "content": "You are a professional translator who translates English financial search queries into Traditional Chinese."},
        {"role": "user", "content": f"Translate this search query into Traditional Chinese: '{query_en}'"}
    ]
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating query: {query_en} -> {e}")
        return ""

# Load and translate queries
with open("questions_example.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

translated = []
for item in tqdm(queries, desc="Translating"):
    zh = translate_with_gpt(item["text_en"])
    translated.append({
        "qid": item["qid"],
        "text_en": item["text_en"],
        "text_zh": zh
    })

with open("translated_queries.json", "w", encoding="utf-8") as f:
    json.dump(translated, f, ensure_ascii=False, indent=2)


In [ ]:
# 🔍 Step 4: Basic BM25 Retrieval (placeholder)
from rank_bm25 import BM25Okapi

# Load passages
import pandas as pd
with open("structured_passages.jsonl", "r") as f:
    passages = [json.loads(line) for line in f]
documents = [p["text"] for p in passages]
bm25 = BM25Okapi([d.split() for d in documents])

# Run retrieval
query = "旅遊保險的保費是多少？"
scores = bm25.get_scores(query.split())
top_k = sorted(zip(scores, passages), reverse=True)[:5]
for score, passage in top_k:
    print(f"[Score: {score:.2f}] {passage['text'][:100]}")

In [ ]:
# 📊 Step 5: Evaluation Setup (Mocked)
ground_truth = {"q1": ["1032_p1_b0"]}
retrieved_pids = [p["pid"] for _, p in top_k]

recall = len(set(retrieved_pids) & set(ground_truth["q1"])) / len(ground_truth["q1"])
print("Recall@5:", recall)